# **Series Temporales**

En este archivo desarrollaremos las siguientes series temporales:
- Time Series Forecasting en Python.
- Modelos estadísticos (AR, ARIMA, SARIMA, Exponential Smoothing).
- Recursive Forecasting (Random Forest, Gradient Boosting Regression).
- Multivariate Forecasting, Ensemble modeling.

Primero importamos todas las librerías que usaremos y las instalamos en caso de ser necesario.

In [2]:
# Instalamos las librerías necesarias
%pip install pandas
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [3]:
# Importamos las librerías necesarias
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

El próximo paso es cargar los datos limpios.

In [4]:
datos = pd.read_csv('../data/equipos_limpio.csv')
datos.head()

,Season,Squad,Country,# Pl,Age,MP,Starts,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR
0,2022-2023,Ajax,Países bajos,19,26.4,6,66,11,9,20,10,1,1,15.0,1.0
1,2022-2023,Atlético Madrid,España,22,28.6,6,66,4,3,7,4,0,2,11.0,0.0
2,2022-2023,Barcelona,España,26,26.4,6,66,12,10,22,12,0,0,9.0,0.0
3,2022-2023,Bayern Munich,Alemania,24,26.6,10,110,21,19,40,20,1,1,20.0,1.0
4,2022-2023,Benfica,Portugal,24,26.0,10,110,25,16,41,20,5,5,19.0,0.0


Veamos la información general de nuestros datos.

In [5]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Season   640 non-null    object 
 1   Squad    640 non-null    object 
 2   Country  640 non-null    object 
 3   # Pl     640 non-null    int64  
 4   Age      640 non-null    float64
 5   MP       640 non-null    int64  
 6   Starts   640 non-null    int64  
 7   Gls      640 non-null    int64  
 8   Ast      640 non-null    int64  
 9   G+A      640 non-null    int64  
 10  G-PK     640 non-null    int64  
 11  PK       640 non-null    int64  
 12  PKatt    640 non-null    int64  
 13  CrdY     576 non-null    float64
 14  CrdR     576 non-null    float64
dtypes: float64(3), int64(9), object(3)
memory usage: 75.1+ KB


Vemos que todavía tenemos algunas filas nulas. Al limpiar los datos no nos importaba tener algunas filas nulas, pero para hacer la clasterización es muy importante no contar con ningún dato de este tipo.

In [6]:
# Eliminamos las filas que contienen valores nulos
datos = datos.dropna()

# Vemos que se ha hecho el cambio correctamente
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 639
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Season   576 non-null    object 
 1   Squad    576 non-null    object 
 2   Country  576 non-null    object 
 3   # Pl     576 non-null    int64  
 4   Age      576 non-null    float64
 5   MP       576 non-null    int64  
 6   Starts   576 non-null    int64  
 7   Gls      576 non-null    int64  
 8   Ast      576 non-null    int64  
 9   G+A      576 non-null    int64  
 10  G-PK     576 non-null    int64  
 11  PK       576 non-null    int64  
 12  PKatt    576 non-null    int64  
 13  CrdY     576 non-null    float64
 14  CrdR     576 non-null    float64
dtypes: float64(3), int64(9), object(3)
memory usage: 72.0+ KB


Observamos que hay algunas variables categóricas que podríamos pasar a numéricas, pues nos podrían ayudar en nuestra predicción posteriormente.

In [7]:
print(datos['Season'].unique())
print(datos['Squad'].unique())
print(datos['Country'].unique())

['2022-2023' '2021-2022' '2020-2021' '2019-2020' '2018-2019' '2016-2017'
 '2014-2015' '2013-2014' '2012-2013' '2011-2012' '2010-2011' '2009-2010'
 '2008-2009' '2007-2008' '2006-2007' '2005-2006' '2004-2005' '2003-2004']
['Ajax' 'Atlético Madrid' 'Barcelona' 'Bayern Munich' 'Benfica' 'Celtic'
 'Chelsea' 'Club Brugge' 'Dinamo Zagreb' 'Dortmund' 'Eint Frankfurt'
 'FC Copenhagen' 'Inter' 'Juventus' 'Leverkusen' 'Liverpool'
 'Maccabi Haifa' 'Manchester City' 'Marseille' 'Milan' 'Napoli'
 'Paris S-G' 'Porto' 'Rangers' 'RB Leipzig' 'RB Salzburg' 'Real Madrid'
 'Sevilla' 'Shakhtar' 'Sporting CP' 'Tottenham' 'Viktoria Plzeň'
 'Atalanta' 'Beşiktaş' 'Dynamo Kyiv' 'Lille' 'Malmö' 'Manchester Utd'
 'Sheriff Tiraspol' 'Villarreal' 'Wolfsburg' 'Young Boys' 'ruZen'
 'Başakşehir' 'Ferencváros' 'Krasnodar' 'Lazio' 'Loko Moscow' "M'Gladbach"
 'Midtjylland' 'Olympiacos' 'frRenn' 'Galatasaray' 'Genk' 'Lyon'
 'Red Star' 'Slavia Prague' 'Valencia' 'AEK Athens' 'CSKA Moscow'
 'Hoffenheim' 'Monaco' 'PSV Eindho

Vemos que hay muchos equipos diferentes, así que no vale la pena convertir esta columna a numérica. Sin embargo, el resto de columas sí que vale la pena convertirlas a numéricas.

In [8]:
# Columna 'Season'
le = LabelEncoder().fit(datos['Season'])
datos['Season'] = le.transform(datos['Season'])

# Obtenemos los valores únicos originales
valores_originales = le.classes_
# Creamos un diccionario de mapeo para ver a qué valor numérico corresponde cada valor original
temporada = dict(zip(valores_originales, le.transform(valores_originales)))

print(temporada)

{'2003-2004': 0, '2004-2005': 1, '2005-2006': 2, '2006-2007': 3, '2007-2008': 4, '2008-2009': 5, '2009-2010': 6, '2010-2011': 7, '2011-2012': 8, '2012-2013': 9, '2013-2014': 10, '2014-2015': 11, '2016-2017': 12, '2018-2019': 13, '2019-2020': 14, '2020-2021': 15, '2021-2022': 16, '2022-2023': 17}


In [9]:
# Columna 'Country'
le = LabelEncoder().fit(datos['Country'])
datos['Country'] = le.transform(datos['Country'])

# Obtenemos los valores únicos originales
valores_originales = le.classes_
# Creamos un diccionario de mapeo para ver a qué valor numérico corresponde cada valor original
paises = dict(zip(valores_originales, le.transform(valores_originales)))

print(paises)

{'Alemania': 0, 'Austria': 1, 'Bielorrusia': 2, 'Bulgaria': 3, 'Bélgica': 4, 'Chipre': 5, 'Croacia': 6, 'Dinamarca': 7, 'Escocia': 8, 'Eslovaquia': 9, 'Eslovenia': 10, 'España': 11, 'Francia': 12, 'Grecia': 13, 'Hungría': 14, 'Inglaterra': 15, 'Israel': 16, 'Italia': 17, 'Moldavia': 18, 'Noruega': 19, 'Países bajos': 20, 'Polonia': 21, 'Portugal': 22, 'República checa': 23, 'Rumanía': 24, 'Rusia': 25, 'Serbia': 26, 'Suecia': 27, 'Suiza': 28, 'Turquía': 29, 'Ucrania': 30}


Tras haber cambiado algunas columnas de categóricas a numéricas, eliminamos aquellas columnas que todavía son categóricas. En nuestro caso es sólo la columna 'Squad'.

In [10]:
# Eliminamos la única columna no numérica que queda
datos = datos.drop(columns=['Squad'])

Comenzamos las Series Temporales separando los datos en entrenamiento y prueba.

In [11]:
# Definimos nuestras variables x e y
x = datos.drop('Gls',axis=1)
y = datos['Gls']

# Dividimos los datos en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


Realizamos los modelos estadísticos de series temporales.

In [12]:
# AR
model_ar = AutoReg(y_train, lags=5)
fit_ar = model_ar.fit()
predictions_ar = fit_ar.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)

# ARIMA
model_arima = ARIMA(y_train, order=(5,1,0))
fit_arima = model_arima.fit()
predictions_arima = fit_arima.forecast(steps=len(y_test))

# SARIMA
model_sarima = SARIMAX(y_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
fit_sarima = model_sarima.fit()
predictions_sarima = fit_sarima.forecast(steps=len(y_test))

# Exponential Smoothing
model_exp = ExponentialSmoothing(y_train, seasonal='add', seasonal_periods=12)
fit_exp = model_exp.fit()
predictions_exp = fit_exp.forecast(steps=len(y_test))

# Modelos de aprendizaje automático
# Random Forest
model_rf = RandomForestRegressor()
model_rf.fit(x_train, y_train)
predictions_rf = model_rf.predict(x_test)

# Gradient Boosting Regression
model_gb = GradientBoostingRegressor()
model_gb.fit(x_train, y_train)
predictions_gb = model_gb.predict(x_test)

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq

Calculamos su error cuadrático medio para evaluar el rendimiento de cada modelo.

In [13]:
# Evaluación del rendimiento
mse_ar = mean_squared_error(y_test, predictions_ar)
mse_arima = mean_squared_error(y_test, predictions_arima)
mse_sarima = mean_squared_error(y_test, predictions_sarima)
mse_exp = mean_squared_error(y_test, predictions_exp)
mse_rf = mean_squared_error(y_test, predictions_rf)
mse_gb = mean_squared_error(y_test, predictions_gb)

print("Error Cuadrático Medio (AR):", mse_ar)
print("Error Cuadrático Medio (ARIMA):", mse_arima)
print("Error Cuadrático Medio (SARIMA):", mse_sarima)
print("Error Cuadrático Medio (Exponential Smoothing):", mse_exp)
print("Error Cuadrático Medio (Random Forest):", mse_rf)
print("Error Cuadrático Medio (Gradient Boosting):", mse_gb)

Error Cuadrático Medio (AR): 47.35889765240159
Error Cuadrático Medio (ARIMA): 48.55417076659892
Error Cuadrático Medio (SARIMA): 49.96725909470113
Error Cuadrático Medio (Exponential Smoothing): 49.722837386139716
Error Cuadrático Medio (Random Forest): 0.5443624277456649
Error Cuadrático Medio (Gradient Boosting): 0.3025083360534547


El Error Cuadrático Medio (ECM) cuantifica el promedio de los errores cuadráticos entre las predicciones del modelo y los valores reales. Un ECM más bajo indica que el modelo tiene una mejor capacidad para predecir los valores reales, mientras que un ECM más alto indica que el modelo tiene una peor capacidad predictiva.

Por lo tanto, comparando los valores de los ECMs para cada modelo, podemos concluir que los mejores modelos (es decir, aquellos que predicen mejor) es el Random Forest y el Gradient Boosting.